In [ ]:
from jupyter_dash import JupyterDash
from dash import dcc
from dash import html
import dash
import dash_leaflet as dl
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State

import base64
import os
import numpy as np
import pandas as pd
from pymongo import MongoClient

import aac_crud
from aac_crud import AnimalShelter





###########################
# Data Manipulation / Model
###########################
# FIX ME change for your username and password and CRUD Python module name
username = "aacuser"
password = "1"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.readAll({}))



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')


image_filename = 'logo.png' # image file name
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(children= [html.B(html.H1('SNHU CS-340 Dashboard')),
                          html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()),
                                   style={'height':'10%', 'width':'10%'})]),
    html.Hr(),
    html.Div([
        dcc.Dropdown(['Water Rescue', 'Mountain/Wilderness Rescue', 'Disaster/Individual Tracking'], value=None, id='dropdown-id'), # create a dropdown that allows the user to sort by Cats and Dogs
        html.Div(children='Justin Winstead SNHU CS-340 MongoDB'), # unique identifier
    ]),
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        page_size=20, # limit results per page
        style_table={'height': 300, 'overflowY': 'auto'} # limit height and enable scrolling
    ),
    html.Br(),
    html.Hr(),
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        dcc.Graph(id='graph'),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

                               
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    # location is set to the latitude and longitude of the first animal in the list
    return [
        dl.Map(style={'width': '1200px', 'height': '400px'}, center=[dff.iloc[1,13],dff.iloc[1,14]], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[dff.iloc[1,13],dff.iloc[1,14]], children=[
                dl.Tooltip(dff.iloc[0,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[1,9])
                ])
            ])
        ])
    ]    

# callback and function for the dropdown sorting
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('dropdown-id', 'value')]) # receives value from the dropdown and uses it as the input to the function
def dropdown_selection(selection):
    # initial condition and condition when selection is cleared
    if selection is None:
        df = pd.DataFrame.from_records(shelter.readAll({}))
        
    # if the user selects Water Rescue from the dropdown
    if selection == 'Water Rescue':
        df = pd.DataFrame(list(shelter.readAll(
            {"breed": { "$in" : ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]},
             "sex_upon_outcome" : "Intact Female",
             "age_upon_outcome_in_weeks" : { "$lt" : 156, "$gt" : 26 }})))
        
    # if the user selects Mountain/Wilderness Rescue from the dropdown
    if selection == 'Mountain/Wilderness Rescue':
        df = pd.DataFrame(list(shelter.readAll(
            {"breed": { "$in" : ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", 
                                 "Siberian Husky", "Rottweiler"]},
             "sex_upon_outcome" : "Intact Male",
             "age_upon_outcome_in_weeks" : { "$lt" : 156, "$gt" : 26 }})))
        
    # if the user selects Disaster/Individual Tracking from the dropdown
    if selection == 'Disaster/Individual Tracking':
        df = pd.DataFrame(list(shelter.readAll(
            {"breed": { "$in" : ["Doberman Pinscher", "German Shepherd", "Golden Retriever",
                                 "Bloodhound", "Rottweiler"]},
             "sex_upon_outcome" : "Intact Male",
             "age_upon_outcome_in_weeks" : { "$lt" : 300, "$gt" : 20 }})))
    
    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    data=df.to_dict('records')
        
    return (data,columns)

@app.callback(
    Output("graph", "figure"),
    [Input('dropdown-id', 'value')]
)
def update_graph(selection):
    # initial condition and condition when selection is cleared
    if selection is None:
        df = pd.DataFrame.from_records(shelter.readAll({}))
        
    # if the user selects Water Rescue from the dropdown
    if selection == 'Water Rescue':
        searchQuery = {"breed": { "$in" : ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]},
             "sex_upon_outcome" : "Intact Female",
             "age_upon_outcome_in_weeks" : { "$lt" : 156, "$gt" : 26 }}
        
        df = pd.DataFrame(list(shelter.readAll(searchQuery)))
        
    # if the user selects Mountain/Wilderness Rescue from the dropdown
    if selection == 'Mountain/Wilderness Rescue':
        searchQuery = {"breed": { "$in" : ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", 
                                 "Siberian Husky", "Rottweiler"]},
             "sex_upon_outcome" : "Intact Male",
             "age_upon_outcome_in_weeks" : { "$lt" : 156, "$gt" : 26 }}
        
        df = pd.DataFrame(list(shelter.readAll(searchQuery)))
        
    # if the user selects Disaster/Individual Tracking from the dropdown
    if selection == 'Disaster/Individual Tracking':
        searchQuery = {"breed": { "$in" : ["Doberman Pinscher", "German Shepherd", "Golden Retriever",
                                 "Bloodhound", "Rottweiler"]},
             "sex_upon_outcome" : "Intact Male",
             "age_upon_outcome_in_weeks" : { "$lt" : 300, "$gt" : 20 }}
        
        df = pd.DataFrame(list(shelter.readAll(searchQuery)))
    
    fig = px.pie(df, names= 'breed')
    fig.update_traces(textposition='inside')
    return fig

app.run()

Success
 * Serving Flask app "SimpleExample" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [14/Aug/2022 17:07:38] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [14/Aug/2022 17:07:39] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [14/Aug/2022 17:07:39] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [14/Aug/2022 17:07:39] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 304 -
127.0.0.1 - - [14/Aug/2022 17:07:39] "GET /_dash-component-suites/dash/dash_table/async-highlight.js HTTP/1.1" 304 -
127.0.0.1 - - [14/Aug/2022 17:07:39] "GET /_dash-component-suites/dash/dash_table/async-table.js HTTP/1.1" 304 -
127.0.0.1 - - [14/Aug/2022 17:07:39] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [14/Aug/2022 17:07:39] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 304 -
127.0.0.1 - - [14/Aug/2022 17:07:39] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [14/Aug/2022 17:07:39] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 